# Scraper componentes: PC Componentes

### 👨‍💻 Autores del proyecto

* [Alejandro Barrionuevo Rosado](https://github.com/Alejandro-BR)
* [Alvaro López Guerrero](https://github.com/Alvalogue72)
* [Andrei Munteanu Popa](https://github.com/andu8705)

Máster de FP en Inteligencia Artifical y Big Data - CPIFP Alan Turing - `Curso 2025/2026`

[pccomponentes](https://www.pccomponentes.com/)

### Explicación de las importaciones
En esta celda se importan las librerías necesarias para el scraping y el manejo de datos. Se utiliza `pandas` para manipulación de datos, `cloudscraper` para evitar bloqueos de Cloudflare, `BeautifulSoup` para parsear HTML, y módulos estándar como `time` y `random`.

In [ ]:
import pandas as pd
import cloudscraper
from bs4 import BeautifulSoup
import time
import random

### Función para limpiar precios
Esta celda define una función que toma una cadena de texto con el precio y la convierte en un valor numérico flotante, eliminando símbolos y adaptando el formato. Es útil para estandarizar los precios extraídos del sitio web.

In [ ]:
def clean_price(price_str):
    """Limpia y convierte una cadena de precio a un float."""
    if price_str:
        price_str = price_str.replace('€', '').replace('$', '').replace(',', '.').strip()
        try:
            return float(price_str)
        except ValueError:
            return None
    return None

### Lista de componentes a scrapear
En esta celda se define una lista con los diferentes tipos de componentes de PC que se desean scrapear del sitio web. Cada elemento representa una categoría de productos.

In [ ]:
components = [
    'placas-base',
    'tarjetas-graficas',
    'procesadores',
    'discos-duros',
    'memorias-ram',
    'cajas-pc',
    'fuentes-alimentacion',
    'refrigeracion-liquida',
    'ventiladores-suplementarios',
    'ventiladores-cpu',
    'tarjetas-sonido', 
]

### Función para scrapear una sola página
Esta celda contiene la función que extrae los productos de una sola página de la categoría indicada. Utiliza BeautifulSoup para analizar el HTML y extraer nombre, precio e imagen de cada producto.

In [ ]:
def scrape_single_page(url, scraper):
    """Función que scrapea una sola página y devuelve la lista de productos encontrados."""
    print(f"Scrapeando: {url}") # Log para ver el progreso
    
    response = scraper.get(url)
    response.encoding = 'utf-8'
    
    products = []
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        items = soup.find_all('div', class_='product-card')
        
        # Señal de que se acabaron las páginas.
        if not items:
            return []
            
        for item in items:
            name_tag = item.find('h3', class_='product-card__title')
            name = name_tag.text.strip() if name_tag else 'Sin nombre'
            
            price_tag = item.find('span', attrs={'data-e2e': 'price-card'})
            price_text = price_tag.text.strip() if price_tag else None
            price_final = clean_price(price_text)
            
            img_tag = item.find('img')
            
            img_url = "Sin imagen"
            if img_tag:
                img_url = img_tag.get('src')
                if img_url and "data:image" in img_url: 
                    img_url = img_tag.get('data-src')

                # Cambio de tamaño 150x150 por 530x530
                if img_url and "w-150-150" in img_url:
                    img_url = img_url.replace("w-150-150", "w-530-530")
            
            products.append({
                'component_type': url.split('/')[-1],
                'name': name,
                'price': price_final,
                'image_url': img_url
            })
    else:
        print(f"Error {response.status_code} en la url {url}")
        
    return products


### Función para scrapear todas las páginas de una categoría
Esta celda define la función que recorre todas las páginas de una categoría de componentes, llamando a la función de scraping de una sola página y acumulando los resultados.

In [ ]:
def scrape_all(base_url, total_pages):
    """Función que scrapea todas las páginas de un componente (url) dado."""
    scraper = cloudscraper.create_scraper()
    all_products = []
    
    for i in range(1, total_pages + 1):
        if i == 1:
            url_actual = base_url
        else:
            url_actual = f"{base_url}?page={i}"
            
        new_products = scrape_single_page(url_actual, scraper)
        
        if not new_products:
            print("No se encontraron más productos. Fin del scraping.")
            break
            
        all_products.extend(new_products)
        
        # Descanso entre 1 y 3 segundos aleatorios para parecer humanos
        rest_time = random.uniform(1, 3)
        time.sleep(rest_time)
        
    return all_products

### Bucle principal para scrapear todas las categorías
En esta celda se recorre la lista de componentes, se construye la URL de cada categoría y se llama a la función de scraping para cada una. Los resultados se acumulan en una lista general.

In [ ]:
all_components = []

for component in components:
    url = f'https://www.pccomponentes.com/{component}'
    final_scrape = scrape_all(url, total_pages=60)
    all_components.extend(final_scrape)
print(f"Total productos extraídos: {len(all_components)}")

Scrapeando: https://www.pccomponentes.com/placas-base
Scrapeando: https://www.pccomponentes.com/placas-base?page=2
Scrapeando: https://www.pccomponentes.com/placas-base?page=3
Scrapeando: https://www.pccomponentes.com/placas-base?page=4
Scrapeando: https://www.pccomponentes.com/placas-base?page=5
Scrapeando: https://www.pccomponentes.com/placas-base?page=6
Scrapeando: https://www.pccomponentes.com/placas-base?page=7
Scrapeando: https://www.pccomponentes.com/placas-base?page=8
Scrapeando: https://www.pccomponentes.com/placas-base?page=9
Scrapeando: https://www.pccomponentes.com/placas-base?page=10
Scrapeando: https://www.pccomponentes.com/placas-base?page=11
Scrapeando: https://www.pccomponentes.com/placas-base?page=12
Scrapeando: https://www.pccomponentes.com/placas-base?page=13
Scrapeando: https://www.pccomponentes.com/placas-base?page=14
Scrapeando: https://www.pccomponentes.com/placas-base?page=15
Scrapeando: https://www.pccomponentes.com/placas-base?page=16
Scrapeando: https://www.

### Guardado de los datos extraídos
En esta celda se crea un DataFrame con los datos obtenidos y se guardan en archivos CSV y JSON para su posterior análisis o uso.

In [ ]:
df_components = pd.DataFrame(all_components)
print(df_components.info())
df_components.to_csv('data/productos_pccomponentes.csv', index=False, encoding='utf-8-sig')
df_components.to_json('data/productos_pccomponentes.json', orient='records', force_ascii=False, indent=4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9055 entries, 0 to 9054
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   component_type  9055 non-null   object 
 1   name            9055 non-null   object 
 2   price           8648 non-null   float64
 3   image_url       9055 non-null   object 
dtypes: float64(1), object(3)
memory usage: 283.1+ KB
None
